### Esse código de Exemplo para utilização do modelo Srgan a partir do sistema web Colaboratory foi baseado no exemplo dado pelo mestre e pesquisador Iga Narendra Pramawijaya

####Importante destacar que para realizar o upscaling a partir deste colaboratory é necessário ter uma conta no google e dar permissão ao colaboratory para acessar seu google drive.

####Além disso será criado em seu google drive uma pasta que contém os arquivos necessário para a utilização do modelo, após a sua utilização a pasta pode ser excluída sem nenhum problema.

1. É necessário para a utilização do modelo o dowload de arquivos específicos que precisam ser acessados pelo modelo, uma solução é usar o próprio drive para armazenar esses arquivos e acessá-los nesse próprio colaboratory

Cabe ressaltar que aqui você está dando permissão para o próprio **google colaboratory**, serviço da própria empresa **Google LLC** acessar seus arquivos no drive.

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive/')

2. Entramos no diretório root(home) do drive.

In [ ]:
%cd /content/drive/MyDrive/

In [ ]:
!ls

3. Vamos criar nosso diretório para armazenar nossos arquivos do modelo. O nome escolhido pelo autor é "SrganNoColab" Mas esse nome pode ser alterado sem problemas.

In [ ]:
from os import path
import os

if path.exists("/content/drive/MyDrive/SrganNoColab") == False:
  os.mkdir("/content/drive/MyDrive/SrganNoColab")

In [ ]:
%cd /content/drive/MyDrive/SrganNoColab

4. Agora com a pasta criada podemos começar o processo de dowload do repositório com os arquivos necessários.

In [ ]:
if path.exists("/content/drive/MyDrive/SrganNoColab/super-resolution-colab") == False:
  !git clone https://github.com/IritaSee/super-resolution-colab.git

In [ ]:
!ls

In [ ]:
%cd super-resolution-colab

###Caso já tenha efetuado a utilização do modelo com esse colab em específico pode pular os proxímos passo e ir direto para o Passo **8**


Agora com o repositório e os arquivos do modelo baixados ainda precisamos baixar os pesos pré-treinados, pois caso não utilizemos pesos que já tenham sido treinados previamente o processo de treinamento iria exigir um longo tempo e seria custoso, considerando ainda que o plano gratuito do google colab não daria suporte de hardware para a realização de tal atividade.

5. Usamos então o comando wget para realizarmos o dowload dos pesos já treinado. Esses pesos estão armazenados no repositório do blog do professor Martin Krasser.


In [ ]:
!wget https://martin-krasser.de/sisr/weights-srgan.tar.gz

6. Agora movemos nosso arquivos de pesos compactados para a pasta weights para podermos extrair nosso arquivo.

In [ ]:
!mv ./weights-srgan.tar.gz ./weights/srgan/weights-srgan.tar.gz

7. Verificamos se nosso arquivo com os pesos está no diretório e extraímos o arquivo no formato tar.gz

In [ ]:
%cd ./weights/srgan/
!ls
!tar -xvf weights-srgan.tar.gz

In [ ]:
%cd ../../

### Agora já baixamos os principais arquivos para o funcionamento do modelo, podemos então definir as principais funções para podermos efetivamente executar as funções de upscaling do nosso modelo.

8. Importamos o generator do nosso modelo *srgan* do arquivo srgan.py

Cabe ressaltar que apenas importamos o generator e mais nenhuma outra função de nosso modelo como um todo, pois como vamos trabalhar com pesos previamente treinados precisamos apenas de nosso generator e alimenta-lo com os pesos já baixados.

Caso fossemos realizar o treinamento outras funções deveriam ser importadas.

In [ ]:
import sys
import matplotlib.pyplot as plt

sys.path.append("/content/drive/MyDrive/SrganNoColab")

from data import DIV2K
from model.srgan import generator

%matplotlib inline

9. agora vamos associar a uma variável os pesos já extraídos do diretório weights/srgan.

Assim podemos a partir dessa variável acessar os arquivos dos pesos que estão formato .h5

In [ ]:
# foi usado o caminho completo por questões de facilidade de visualização
diretoriosPesos = "/content/drive/MyDrive/SrganNoColab/super-resolution-colab/weights/srgan/weights/srgan"
arquivoPesos = lambda filename: os.path.join(diretoriosPesos, filename)

os.makedirs(diretoriosPesos, exist_ok=True)

10. Agora podemos instanciar nosso generator na variável generator_model e utilizando a função load_weights, função essa do próprio generator, alimentar nosso generator com os pesos já pré treinados.

In [ ]:
pre_generator = generator()
gan_generator = generator()

gan_generator.load_weights(arquivoPesos("gan_generator.h5"))
pre_generator.load_weights(arquivoPesos("pre_generator.h5"))

# generator_model = generator()
# generator_model.load_weights(arquivoPesos("gan_generator.h5"))

####Agora com o generator já instânciado e alimentado e instânciado criamos a função que vai receber uma imagem de baixa resolução como input e usando a função do generator *resolve_single*, vamos a partir dessa imagem gerar sua versão em maior resolução.


11. Além do upscaling em sí, vamos usar as bibliotecas do matplotlib para podermos apresentar a imagem original e sua versão após o processo. Isso tudo será implementado na função chamada *upscaling_visualizacao*

In [ ]:
from model import resolve_single
from utils import load_image

def upscaling_visualizacao(lr_image):
  lr = load_image(lr_image)

  pre_upscaling = resolve_single(pre_generator, lr)
  gan_upscaling = resolve_single(gan_generator, lr)

  plt.figure(figsize=(5,5))

  images= [lr, gan_upscaling]
  titles = ["Imagem Original em baixa resolução", "Imagem após o processo de superresolution completo"]
  positions = [1, 2]

  for i, (img, titles, pos) in enumerate(zip(images, titles, positions)):
    plt.subplot(2, 2, pos)
    plt.title(titles)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

  return gan_upscaling

12. Existem Casos onde a imagem selecionada acaba por não seguir o modelo RxGxB, podendo então haver um outro canal na imagem chamado canal alfa, nesse caso precisamos criar uma função para converter essa imagem antes de realizar o processo de upscaling, removendo esse canal alfa. Para isso usamos a biblioteca numpy para transformar os dados da imagem em um array que vai ter 4 canais (alfa, Red, Green, Blue) e a biblioteca PIL para a partir desse array convertermos em um RGB (Red, Green, Blue).

In [ ]:
from PIL import Image
import numpy as np

# Usamos essa função, pois precisamos converter o array da imagem para o formato de imagem RGB
def convert_upscaling(lr_image):
  lr = np.array(Image.open(lr_image).convert("RGB"))

  pre_upscaling = resolve_single(pre_generator, lr)
  gan_upscaling = resolve_single(gan_generator, lr)

  plt.figure(figsize=(10,10))

  images= [lr, pre_upscaling, gan_upscaling]
  titles = ["Imagem Original em baixa resolução", "Imagem antes do processo de superresolution completo", "Imagem após o processo de superresolution completo"]
  positions = [1, 3, 4]

  for i, (img, titles, pos) in enumerate(zip(images, titles, positions)):
    plt.subplot(2, 2, pos)
    plt.title(titles)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

  return gan_upscaling

13. Vamos Então Fazer a Leitura do aquivo de imagem escolhido todas as imagens usadas para o superResolution estarão no diretório

 /SrganNoColab/super-resolution-colab/demo

In [ ]:
import cv2
from PIL import Image

imgToCV = files.upload()
if imgToCV is not None:
  for filename in imgToCV.keys():
    with open(os.path.join("/content/drive/MyDrive/SrganNoColab/super-resolution-colab/demo", filename), "wb") as f:
      f.write(imgToCV[filename])
    pilImage = Image.open(filename).convert("RGB")
    TransformCv = np.array(pilImage)[:, :, ::-1].copy()
    # imageUploaded = Image.open(filename)
    # display(imageUploaded)

cv2.imwrite("image.jpg", TransformCv)
ImagemFinal = cv2.imread("image.jpg")

ganImage = convert_upscaling("image.jpg")


# upscaling_visualizacao("image.jpg")

In [ ]:
# agora preciso salvar a cópia da imagem de sáida(SR) em um arquivo, colocar para esse arquivo ser salvo localmente.

14. Caso deseje salvar a imagem em alta resolução com adição de **borda** é só executar essa celúla

A adição da borda branca é feita para facilitar a visualização da diferença de resolução entre a imagem antes e depois do SR, pois quanto mais zoom in na imagem que passou pelo processo de SR, mais será evidente os traços de deformidades que o processo deixa na imagem.

In [ ]:
if ganImage is not None:
  ganImage = np.array(ganImage)
  # cv2.imwrite("image.jpg", ganImage)
  border = cv2.copyMakeBorder(ganImage, 600, 600, 600, 600, cv2.BORDER_CONSTANT, value=[255,255,255])
  cv2.imwrite("image.jpg", border)
  files.download("image.jpg")

15. Caso deseje salvar a imagem em alta resolução **sem** adição de **borda** é só executar essa celúla


In [ ]:
if ganImage is not None:
  ganImage = np.array(ganImage)
  # cv2.imwrite("image.jpg", ganImage)
  cv2.imwrite("image.jpg", ganImage)
  files.download("image.jpg")